In [ ]:
import numpy as np
import cv2

# variables that you can adjust
driveName = r"\\.\PhysicalDrive1"  # Change this to the correct drive
sectorSize = 512    # The number of bytes in a sector (two hexidecimal numbers is 8 bits, or a byte)
startingSector = 1024
frameWidth = 384    # The number of pixels in the width (Each pixel is a byte)
frameHeight = 296    # The number of pixels in the height
numFrames2Read = 90
fps = 20

f = open(driveName, "rb")  # Open drive

# TO DO: Seems like the starting sector isn't correct. Try to search for the write key location. : nvm. Got it.
# Write_Key = 0x0D7CBA17
f.seek(startingSector * sectorSize)  # Move to correct sector

# Gather frames for a video
frame_array = []

for numFrame in range(numFrames2Read):  # Loop through frames
    rawBytes = f.read(frameWidth * frameHeight)
    pixelValues = list(rawBytes)  # Converts from bytes to list. This and the next line can be done more efficiently
    
    DMAComplete = pixelValues[-16]
    Timestamp = pixelValues[-8:-4]
    Timestamp = Timestamp[0] + 16 * Timestamp[1] + pow(16, 2) * Timestamp[2] + pow(16, 3) * Timestamp[3]
    Line_Count = pixelValues[-12:-8]
    Frame_Number = pixelValues[-4:]
    
    frame = np.array(pixelValues) / 255.0  # Convert to numpy array
    frame = np.reshape(frame, (frameHeight, frameWidth))  # reshape into correct frame dimensions
    
    frame_array.append(frame)
    
    cv2.imshow("sdCard Video", frame)  # Display frame
    cv2.waitKey(
        0
    )  # Wait for button press to show next frame. Can use time.sleep(n) to increment frame every 'n' seconds
    

    print("\nDMA Complete: ", DMAComplete, 
          "\nTimestamp: ", Timestamp, 
          "\n# of Lines: ", Line_Count, 
          "\nFrame Number: ", Frame_Number)


# cv2.destroyAllWindows()
print(frame_array[1])
print(len(frame_array[1]), len(frame_array[1][1]))

# cv2.imshow("SD", frame_array[1])

f.close()

In [5]:
out = cv2.VideoWriter('SD_vid.avi', cv2.VideoWriter_fourcc('P','V','W','2'), 20, (frameWidth,frameHeight), 0)
 
for i in range(len(frame_array)):
    out.write(frame_array[i])

out.release()